# GERADOR DE POEMAS COM IA E FEEDBACK

## Objetivo do Sistema

Este projeto implementa um **sistema completo de geração de poemas** usando IA generativa via API, com as seguintes funcionalidades:

- **Geração de poemas**: Uso de LLM (Large Language Model) via API
- **Feedback do usuário**: Coleta estruturada de avaliação do poema
- **Novo Chat vs Chat Contínuo**: Diferenciação entre iniciar nova conversa ou continuar com histórico
- **Interface interativa**: Widgets no Jupyter para experiência intuitiva
- **Melhoria automática**: Refinamento de prompts baseado no feedback

## Responsabilidades

- **Grupo 1**: Integração com LLM e chamadas à API
- **Grupo 2**: Interface interativa com widgets
- **Grupo 3**: Sistema de feedback e refinamento de prompts

---

In [ ]:
!pip install gradio==5.40.0
!pip install -U -q "google-genai"

In [ ]:
# SISTEMA COMPLETO DE FEEDBACK (Grupo 3)
# Importações necessárias para o sistema avançado
import difflib
from collections import Counter

# 1. VALIDAÇÃO E MAPEAMENTO INTELIGENTE DE REFINAMENTOS
def mapear_alvo_inteligente(alvo, poema_atual):
    """
    Mapeia descrições naturais (ex: "terceira estrofe") para texto real do poema.
    
    Args:
        alvo (str): Descrição do alvo (ex: "primeira linha", "segunda estrofe")
        poema_atual (str): Poema atual
    
    Returns:
        str: Texto real encontrado no poema ou alvo original
    """
    alvo_lower = alvo.lower().strip()
    linhas = [linha.strip() for linha in poema_atual.split('\n') if linha.strip()]
    
    # Separar em estrofes (grupos de linhas separadas por linha vazia)
    estrofes = []
    estrofe_atual = []
    
    for linha in poema_atual.split('\n'):
        if linha.strip():
            estrofe_atual.append(linha.strip())
        else:
            if estrofe_atual:
                estrofes.append('\n'.join(estrofe_atual))
                estrofe_atual = []
    if estrofe_atual:
        estrofes.append('\n'.join(estrofe_atual))
    
    # Mapeamento por posição numérica
    if 'primeira' in alvo_lower and 'linha' in alvo_lower:
        return linhas[0] if linhas else alvo
    elif 'segunda' in alvo_lower and 'linha' in alvo_lower:
        return linhas[1] if len(linhas) > 1 else alvo
    elif 'terceira' in alvo_lower and 'linha' in alvo_lower:
        return linhas[2] if len(linhas) > 2 else alvo
    elif 'última' in alvo_lower and 'linha' in alvo_lower:
        return linhas[-1] if linhas else alvo
    
    # Mapeamento por estrofe
    elif 'primeira' in alvo_lower and 'estrofe' in alvo_lower:
        return estrofes[0] if estrofes else alvo
    elif 'segunda' in alvo_lower and 'estrofe' in alvo_lower:
        return estrofes[1] if len(estrofes) > 1 else alvo
    elif 'terceira' in alvo_lower and 'estrofe' in alvo_lower:
        return estrofes[2] if len(estrofes) > 2 else alvo
    elif 'última' in alvo_lower and 'estrofe' in alvo_lower:
        return estrofes[-1] if estrofes else alvo
    
    # Mapeamento por números ordinais
    for i, num in enumerate(['primeiro', 'segunda', 'terceiro', 'quarto', 'quinto'], 1):
        if num in alvo_lower:
            if 'linha' in alvo_lower and len(linhas) >= i:
                return linhas[i-1]
            elif 'estrofe' in alvo_lower and len(estrofes) >= i:
                return estrofes[i-1]
    
    # Se não encontrou mapeamento, retorna o alvo original
    return alvo

def validar_refinamento(refinamento, poema_atual):
    """
    Valida se o refinamento faz sentido no contexto do poema.
    Agora com mapeamento inteligente de alvos.
    
    Args:
        refinamento (dict): {'acao': str, 'alvo': str, 'novo_conteudo': str}
        poema_atual (str): Poema atual
    
    Returns:
        dict: {'valido': bool, 'erro': str (se inválido), 'alvo_mapeado': str}
    """
    alvo_original = refinamento['alvo']
    acao = refinamento['acao']
    
    # Mapear alvo para texto real
    alvo_mapeado = mapear_alvo_inteligente(alvo_original, poema_atual)
    
    # Verifica se o alvo existe (para Editar e Remover)
    if acao in ['Editar', 'Remover']:
        # Primeiro tenta encontrar o alvo mapeado
        if alvo_mapeado.lower() in poema_atual.lower():
            # Encontrou o texto mapeado
            pass
        else:
            # Tenta busca por palavras-chave do alvo original
            palavras_alvo = alvo_original.lower().split()
            poema_lower = poema_atual.lower()
            
            encontrado = any(palavra in poema_lower for palavra in palavras_alvo if len(palavra) > 3)
            
            if not encontrado:
                return {
                    'valido': False,
                    'erro': f"Alvo '{alvo_original}' não encontrado no poema. Tente ser mais específico ou copiar o texto exato.",
                    'alvo_mapeado': alvo_mapeado
                }
    
    # Verifica se há novo conteúdo quando necessário
    if acao in ['Adicionar', 'Editar']:
        if not refinamento.get('novo_conteudo'):
            return {
                'valido': False,
                'erro': f"Ação '{acao}' precisa de novo conteúdo",
                'alvo_mapeado': alvo_mapeado
            }
    
    return {'valido': True, 'alvo_mapeado': alvo_mapeado}

# 2. DETECÇÃO DE CONFLITOS
def detectar_conflitos(refinamentos):
    """
    Detecta conflitos entre refinamentos.
    
    Args:
        refinamentos (list): Lista de refinamentos
    
    Returns:
        list: Lista de conflitos detectados
    """
    conflitos = []
    
    for i, ref1 in enumerate(refinamentos):
        for j, ref2 in enumerate(refinamentos[i+1:], start=i+1):
            # Mesmo alvo com ações diferentes
            if ref1['alvo'].lower() == ref2['alvo'].lower():
                if ref1['acao'] != ref2['acao']:
                    conflitos.append({
                        'tipo': 'mesmo_alvo_acoes_diferentes',
                        'indices': [i, j],
                        'descricao': f"Refinamentos {i+1} e {j+1} têm o mesmo alvo com ações diferentes",
                        'sugestao': 'Mesclar em um único refinamento ou remover um deles'
                    })
            
            # Remover seguido de editar
            if ref1['acao'] == 'Remover' and ref2['acao'] == 'Editar':
                if ref1['alvo'].lower() in ref2['alvo'].lower():
                    conflitos.append({
                        'tipo': 'editar_apos_remover',
                        'indices': [i, j],
                        'descricao': f"Refinamento {j+1} tenta editar algo que será removido",
                        'sugestao': 'Remover o refinamento de edição ou mudar a ordem'
                    })
    
    return conflitos

# 3. PRIORIZAÇÃO DE REFINAMENTOS
def priorizar_refinamentos(refinamentos):
    """
    Ordena refinamentos na ordem ideal: Remover → Editar → Adicionar
    
    Args:
        refinamentos (list): Lista de refinamentos
    
    Returns:
        list: Refinamentos ordenados
    """
    prioridades = {
        'Remover': 1,
        'Editar': 2,
        'Adicionar': 3
    }
    
    return sorted(refinamentos, key=lambda r: prioridades.get(r['acao'], 999))

# 4. TEMPLATES ESPECIALIZADOS
TEMPLATES_REFINAMENTO = {
    'Editar': """POEMA ORIGINAL:
{poema}

INSTRUÇÃO MUITO ESPECÍFICA: 
Você deve EDITAR APENAS a parte "{alvo}" para ficar "{novo_conteudo}".

REGRAS OBRIGATÓRIAS:
1. COPIE o poema original EXATAMENTE como está
2. Modifique SOMENTE a parte mencionada: "{alvo}"
3. TODO o resto deve permanecer IDÊNTICO palavra por palavra
4. NÃO reescreva o poema inteiro - apenas ajuste a parte específica
5. Mantenha quebras de linha, pontuação e formatação originais
6. Responda APENAS com o poema modificado, sem comentários

IMPORTANTE: É uma pequena modificação, não um novo poema!""",
    
    'Adicionar': """POEMA ORIGINAL:
{poema}

INSTRUÇÃO MUITO ESPECÍFICA:
Você deve ADICIONAR "{novo_conteudo}" na posição "{alvo}".

REGRAS OBRIGATÓRIAS:
1. MANTENHA todo o poema original exatamente igual
2. ADICIONE o novo conteúdo na posição indicada
3. NÃO modifique nada que já existe
4. Integre naturalmente sem quebrar o fluxo
5. Preserve formatação, quebras de linha e estilo
6. Responda APENAS com o poema completo (original + adição)

IMPORTANTE: É uma adição ao poema existente, não um novo poema!""",
    
    'Remover': """POEMA ORIGINAL:
{poema}

INSTRUÇÃO MUITO ESPECÍFICA:
Você deve REMOVER completamente a parte: "{alvo}".

REGRAS OBRIGATÓRIAS:
1. MANTENHA todo o resto do poema exatamente igual
2. REMOVA apenas o que foi especificado: "{alvo}"
3. Ajuste transições se necessário para manter fluidez
4. NÃO reescreva outras partes
5. Preserve formatação e estilo original
6. Responda APENAS com o poema sem a parte removida

IMPORTANTE: É uma remoção específica, não um novo poema!"""
}

def gerar_prompt_refinamento(refinamento, poema_atual):
    """
    Gera prompt otimizado baseado no tipo de refinamento.
    Usa estratégia de preservação forçada para evitar reescrita completa.
    
    Args:
        refinamento (dict): Refinamento a aplicar
        poema_atual (str): Poema atual
    
    Returns:
        str: Prompt formatado
    """
    acao = refinamento['acao']
    alvo = refinamento['alvo']
    novo_conteudo = refinamento.get('novo_conteudo', '')
    
    # Estratégia mais agressiva para forçar preservação
    if acao == 'Editar':
        # Quebra o poema em linhas para análise
        linhas = poema_atual.split('\n')
        linhas_numeradas = '\n'.join([f"LINHA {i+1}: {linha}" for i, linha in enumerate(linhas)])
        
        prompt = f"""TAREFA CRÍTICA DE EDIÇÃO PRECISA:

POEMA ORIGINAL COM NUMERAÇÃO (PRESERVE EXATAMENTE):
{linhas_numeradas}

MODIFICAÇÃO SOLICITADA:
- Encontre e modifique apenas: "{alvo}"
- Aplicar mudança: "{novo_conteudo}"

INSTRUÇÕES OBRIGATÓRIAS:
1. COPIE todas as linhas exatamente como estão
2. Modifique SOMENTE a parte que contém "{alvo}"
3. Mantenha numeração e estrutura idênticas
4. Se "{alvo}" se refere a uma estrofe/linha específica, modifique apenas ela
5. TODO o resto deve ser uma CÓPIA EXATA do original

FORMATO DE RESPOSTA: Retorne apenas o poema modificado (sem numeração), preservando quebras de linha originais.

CRÍTICO: Esta é uma pequena correção, não um novo poema!"""

    elif acao == 'Adicionar':
        prompt = f"""TAREFA DE ADIÇÃO CONTROLADA:

POEMA ORIGINAL (MANTENHA INTACTO):
{poema_atual}

ADIÇÃO SOLICITADA:
- Posição: {alvo}
- Conteúdo a adicionar: "{novo_conteudo}"

INSTRUÇÕES:
1. PRESERVE todo o poema original PALAVRA POR PALAVRA
2. Adicione o novo conteúdo na posição indicada
3. Mantenha formatação e quebras de linha originais
4. NÃO modifique nenhuma palavra existente

RETORNE: O poema original + a adição (sem comentários)"""

    else:  # Remover
        prompt = f"""TAREFA DE REMOÇÃO ESPECÍFICA:

POEMA ORIGINAL:
{poema_atual}

REMOVER: "{alvo}"

INSTRUÇÕES:
1. MANTENHA tudo igual exceto a parte a ser removida
2. Remova apenas "{alvo}" especificamente
3. Ajuste transições se necessário
4. Preserve formatação original

RETORNE: Poema sem a parte removida (sem comentários)"""
    
    return prompt

# 5. AVALIAÇÃO DE RESULTADOS
def avaliar_refinamento(poema_antes, poema_depois, refinamento):
    """
    Avalia se o refinamento foi aplicado com sucesso.
    
    Args:
        poema_antes (str): Poema antes do refinamento
        poema_depois (str): Poema depois do refinamento
        refinamento (dict): Refinamento aplicado
    
    Returns:
        dict: Métricas de avaliação
    """
    # Calcula diferenças
    diff = list(difflib.ndiff(poema_antes.splitlines(), poema_depois.splitlines()))
    mudancas = [linha for linha in diff if linha.startswith('+ ') or linha.startswith('- ')]
    
    # Análise básica
    resultado = {
        'mudancas_detectadas': len(mudancas) > 0,
        'num_linhas_mudadas': len(mudancas),
        'tipo_refinamento': refinamento['acao'],
        'alvo': refinamento['alvo']
    }
    
    # Verificação específica por tipo
    alvo_lower = refinamento['alvo'].lower()
    poema_depois_lower = poema_depois.lower()
    
    if refinamento['acao'] == 'Remover':
        # Sucesso se o alvo NÃO está mais presente
        resultado['sucesso'] = alvo_lower not in poema_depois_lower
        resultado['motivo'] = 'Alvo removido' if resultado['sucesso'] else 'Alvo ainda presente'
        
    elif refinamento['acao'] == 'Adicionar':
        # Sucesso se o novo conteúdo FOI adicionado
        novo_conteudo_lower = refinamento.get('novo_conteudo', '').lower()
        resultado['sucesso'] = novo_conteudo_lower in poema_depois_lower
        resultado['motivo'] = 'Conteúdo adicionado' if resultado['sucesso'] else 'Conteúdo não encontrado'
        
    else:  # Editar
        # Sucesso se houve mudanças
        resultado['sucesso'] = len(mudancas) > 0
        resultado['motivo'] = 'Poema modificado' if resultado['sucesso'] else 'Nenhuma mudança detectada'
    
    return resultado

# 6. SUGESTÕES AUTOMÁTICAS
def sugerir_refinamentos(poema):
    """
    Analisa o poema e sugere possíveis refinamentos.
    
    Args:
        poema (str): Poema a analisar
    
    Returns:
        list: Lista de sugestões
    """
    sugestoes = []
    
    # Análise de tamanho
    versos = [v for v in poema.strip().split('\n') if v.strip()]
    if len(versos) > 20:
        sugestoes.append({
            'tipo': 'estrutural',
            'prioridade': 'média',
            'sugestao': f'Poema longo ({len(versos)} versos). Considere encurtar.',
            'refinamento_sugerido': {
                'acao': 'Remover',
                'alvo': 'versos excessivos',
                'novo_conteudo': ''
            }
        })
    
    # Análise de repetição
    palavras = poema.lower().split()
    freq = Counter(p for p in palavras if len(p) > 4)
    repetidas = [(p, c) for p, c in freq.items() if c > 3]
    
    if repetidas:
        palavra, count = repetidas[0]
        sugestoes.append({
            'tipo': 'estilístico',
            'prioridade': 'baixa',
            'sugestao': f'Palavra "{palavra}" repetida {count} vezes',
            'refinamento_sugerido': {
                'acao': 'Editar',
                'alvo': f'palavra {palavra}',
                'novo_conteudo': 'usar sinônimos'
            }
        })
    
    # Análise de versos muito curtos
    versos_curtos = [v for v in versos if len(v.split()) < 3]
    if len(versos_curtos) > len(versos) * 0.3:
        sugestoes.append({
            'tipo': 'estrutural',
            'prioridade': 'média',
            'sugestao': 'Muitos versos curtos. Considere expandir.',
            'refinamento_sugerido': {
                'acao': 'Editar',
                'alvo': 'versos curtos',
                'novo_conteudo': 'expandir com mais detalhes'
            }
        })
    
    return sugestoes

# 7. FUNÇÃO PRINCIPAL DE PROCESSAMENTO
def processar_refinamentos(refinamentos, poema_atual):
    """
    Pipeline completo de processamento de refinamentos.
    
    Args:
        refinamentos (list): Lista de refinamentos
        poema_atual (str): Poema atual
    
    Returns:
        dict: Resultado do processamento
    """
    # 1. Validar cada refinamento e mapear alvos
    erros = []
    refinamentos_mapeados = []
    
    for i, ref in enumerate(refinamentos):
        validacao = validar_refinamento(ref, poema_atual)
        if not validacao['valido']:
            erros.append(f"Refinamento {i+1}: {validacao['erro']}")
        else:
            # Usar o alvo mapeado no refinamento
            ref_mapeado = ref.copy()
            ref_mapeado['alvo'] = validacao['alvo_mapeado']
            ref_mapeado['alvo_original'] = ref['alvo']  # Guardar o original para logs
            refinamentos_mapeados.append(ref_mapeado)
    
    if erros:
        return {'sucesso': False, 'erros': erros}
    
    # Usar os refinamentos mapeados daqui em diante
    refinamentos = refinamentos_mapeados
    
    # 2. Detectar conflitos
    conflitos = detectar_conflitos(refinamentos)
    if conflitos:
        avisos = [f"{c['descricao']}: {c['sugestao']}" for c in conflitos]
        # Continua mesmo com conflitos, mas avisa
    else:
        avisos = []
    
    # 3. Priorizar ordem
    refinamentos_ordenados = priorizar_refinamentos(refinamentos)
    
    # 4. Gerar prompts otimizados
    prompts = []
    for ref in refinamentos_ordenados:
        prompt = gerar_prompt_refinamento(ref, poema_atual)
        prompts.append({
            'refinamento': ref,
            'prompt': prompt
        })
    
    return {
        'sucesso': True,
        'avisos': avisos,
        'refinamentos_ordenados': refinamentos_ordenados,
        'prompts': prompts,
        'ordem_aplicacao': [r['acao'] for r in refinamentos_ordenados]
    }

print("Sistema Completo de Feedback (Grupo 3) carregado!")
print("Funções disponíveis: validar_refinamento, detectar_conflitos, priorizar_refinamentos")
print("Templates: gerar_prompt_refinamento, avaliar_refinamento, sugerir_refinamentos")
print("Pipeline principal: processar_refinamentos")

In [ ]:
# CONFIGURAÇÃO DA CHAVE API DO GOOGLE GEMINI PARA NOTEBOOK 
import os

# OPÇÃO 1: Substitua "SUA_CHAVE_AQUI" pela sua chave real do Google AI Studio
# Obtenha sua chave em: https://aistudio.google.com/
os.environ["GEMINI_API_KEY"] = "SUA_CHAVE_AQUI"

print("🔑 Configuração da API:")
if os.environ.get("GEMINI_API_KEY") and os.environ.get("GEMINI_API_KEY") != "SUA_CHAVE_AQUI":
    print("✅ Chave API configurada com sucesso!")
    print(f"🔗 Chave carregada: {os.environ.get('GEMINI_API_KEY')[:10]}...")
else:
    print("ATENÇÃO: Substitua 'SUA_CHAVE_AQUI' pela sua chave real!")
    print("Obtenha sua chave em: https://aistudio.google.com/")

In [ ]:
# Importações
import gradio as gr
from google import genai

# --- Configuração da API do Gemini ---
try:
    client = genai.Client()
    model_name = "gemini-2.5-pro"
except Exception as e:
    print(f"Erro ao configurar a API do Gemini: {e}")
    print("Certifique-se de ter adicionado sua GEMINI_API_KEY como um Secret no Kaggle.")
    client = None
    model_name = None


# --- Função de Backend com Sistema Avançado ---
def gerar_poema(tema, estilo, tamanho, temperatura, poema_atual, refinamentos_list, chat_atual):
    """
    Gera ou refina um poema usando o SISTEMA AVANÇADO do Grupo 3 quando há refinamentos.
    Mantém o mesmo chat para refinamentos ou cria novo para poemas novos.
    """
    temp_normalizada = temperatura / 100.0
    system_instruction = "Você é um poeta de renome, especialista em vários estilos. Sua única saída deve ser o poema (refinado ou gerado) em português, sem explicações, saudações ou markdown adicional."

    # Verificar se há refinamentos para usar sistema avançado
    if refinamentos_list and len(refinamentos_list) > 0:
        print(f"🔄 Aplicando {len(refinamentos_list)} refinamentos usando sistema avançado...")
        
        # USAR SISTEMA AVANÇADO DO GRUPO 3
        resultado_processamento = processar_refinamentos(refinamentos_list, poema_atual)
        
        if not resultado_processamento['sucesso']:
            erros = "\n".join(resultado_processamento['erros'])
            return f"❌ Erros nos refinamentos:\n{erros}", poema_atual, gr.Tabs(selected=1), chat_atual
        
        # Avisos sobre conflitos (se houver)
        if resultado_processamento['avisos']:
            avisos = "\n".join(resultado_processamento['avisos'])
            print(f"⚠️ Avisos detectados:\n{avisos}")
        
        # Aplicar cada refinamento na ordem otimizada
        poema_refinado = poema_atual
        
        for prompt_info in resultado_processamento['prompts']:
            refinamento = prompt_info['refinamento']
            prompt_otimizado = prompt_info['prompt']
            
            print(f"🔄 Aplicando: {refinamento['acao']} - {refinamento['alvo']}")
            
            try:
                # Usar chat existente para refinamentos
                if chat_atual is None:
                    chat_atual = client.chats.create(model=model_name, config={
                        "temperature": temp_normalizada,
                        "system_instruction": system_instruction
                    })
                
                response = chat_atual.send_message(prompt_otimizado)
                
                if response.text:
                    poema_antes = poema_refinado
                    poema_candidato = response.text.strip()
                    
                    # VALIDAÇÃO EXTRA: Verificar se preservou o original
                    if refinamento['acao'] in ['Editar', 'Adicionar']:
                        # Calcular similaridade entre original e resultado
                        palavras_originais = set(poema_antes.lower().split())
                        palavras_resultado = set(poema_candidato.lower().split())
                        
                        # Se perdeu mais de 30% das palavras originais, provavelmente reescreveu
                        palavras_preservadas = len(palavras_originais & palavras_resultado)
                        percentual_preservacao = palavras_preservadas / len(palavras_originais) if palavras_originais else 0
                        
                        if percentual_preservacao < 0.7:  # Menos de 70% preservado
                            print(f"⚠️ ATENÇÃO: {refinamento['acao']} pode ter reescrito o poema (preservação: {percentual_preservacao:.1%})")
                            print("🔄 Tentando abordagem mais conservadora...")
                            
                            # Prompt mais agressivo
                            prompt_conservador = f"""EMERGÊNCIA - CORREÇÃO SIMPLES:
Original: {poema_antes}

Faça APENAS esta mudança: {refinamento['acao']} "{refinamento['alvo']}" para "{refinamento.get('novo_conteudo', '')}"

COPIE o texto original e mude SÓ essa parte específica. Não reescreva!"""
                            
                            response2 = chat_atual.send_message(prompt_conservador)
                            if response2.text:
                                poema_candidato = response2.text.strip()
                    
                    poema_refinado = poema_candidato
                    
                    # Avaliar se o refinamento funcionou
                    avaliacao = avaliar_refinamento(poema_antes, poema_refinado, refinamento)
                    
                    if avaliacao['sucesso']:
                        print(f"✅ {refinamento['acao']} aplicado com sucesso: {avaliacao['motivo']}")
                    else:
                        print(f"⚠️ {refinamento['acao']} pode não ter funcionado: {avaliacao['motivo']}")
                else:
                    print(f"❌ Falha ao aplicar refinamento: {refinamento['acao']}")
                    
            except Exception as e:
                print(f"❌ Erro ao aplicar refinamento '{refinamento['acao']}': {e}")
                continue
        
        # Sugestões automáticas (opcional)
        sugestoes = sugerir_refinamentos(poema_refinado)
        if sugestoes:
            print(f"\n💡 {len(sugestoes)} sugestões automáticas detectadas:")
            for sug in sugestoes[:2]:  # Mostrar apenas as 2 primeiras
                print(f"   - {sug['sugestao']}")
        
        return poema_refinado, poema_refinado, gr.Tabs(selected=1), chat_atual
        
    else:
        # GERAÇÃO INICIAL (sem refinamentos)
        prompt = f"Crie um poema {estilo} sobre: {tema}."
        if tamanho == "Curto (1-2 estrofes)":
            prompt += " O poema deve ter 1 ou 2 estrofes."
        elif tamanho == "Médio (3-4 estrofes)":
            prompt += " O poema deve ter 3 ou 4 estrofes."
        else:
            prompt += " O poema deve ter 5 ou mais estrofes."

    if tema == "":
        return f"⚠️ Defina um tema para o poema", poema_atual, gr.Tabs(selected=1), chat_atual

    if client and model_name:
        try:
            # Criar novo chat para poema novo
            chat = client.chats.create(model=model_name, config={
                "temperature": temp_normalizada,
                "system_instruction": system_instruction
            })

            response = chat.send_message(prompt)

            if response.text:
                print("✅ Poema gerado com sucesso!")
                return response.text, response.text, gr.Tabs(selected=1), chat
            else:
                return "⚠️ Não foi possível gerar o poema. Tente novamente.", poema_atual, gr.Tabs(selected=1), chat
        except Exception as e:
            error_message = f"Erro na API do Gemini: {e}"
            print(error_message)
            return f"❌ {error_message}", poema_atual, gr.Tabs(selected=1), chat_atual

    return "❌ API não configurada.", poema_atual, gr.Tabs(selected=1), None


# --- CSS Personalizado ---
custom_css = """
.gradio-container {
    max-width: 900px !important;
    margin: auto !important;
}

h1 {
    text-align: center;
    color: #2c3e50;
    font-size: 2.5em;
    margin-bottom: 10px;
}

.subtitle {
    text-align: center;
    color: #7f8c8d;
    font-size: 1.2em;
    margin-bottom: 30px;
}

.generate-btn {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%) !important;
    border: none !important;
    color: white !important;
    font-size: 1.1em !important;
    padding: 15px 30px !important;
    border-radius: 10px !important;
    font-weight: bold !important;
    transition: transform 0.2s !important;
}

.generate-btn:hover {
    transform: scale(1.05) !important;
}

.exemplo-btn {
    background: #3498db !important;
    color: white !important;
    border-radius: 8px !important;
}

.limpar-btn {
    background: #95a5a6 !important;
    color: white !important;
    border-radius: 8px !important;
}

.button-row {
    display: flex !important;
    justify-content: space-between !important;
    gap: 10px !important;
}

.refinamento-box {
    border: 2px solid #667eea;
    border-radius: 10px;
    padding: 15px;
    margin: 10px 0;
    background: #f8f9fa;
}
"""

# --- Interface Gradio ---
with gr.Blocks(theme=gr.themes.Soft(), css=custom_css, title="Gerador de Poemas IA") as demo:
    poema_state = gr.State(value="")
    refinamentos_state = gr.State(value=[])  # Lista de refinamentos
    chat_state = gr.State(value=None)  # Armazena o objeto chat

    gr.Markdown("# Gerador de Poemas com IA")
    gr.Markdown("<p class='subtitle'>✨ Transforme suas ideias em versos únicos e inspiradores</p>")

    with gr.Tabs() as tabs:
        # ABA 1: Definição do Tema
        with gr.Tab("📝 Criar Poema", id=0):
            gr.Markdown("### Defina o tema e estilo do seu poema")

            with gr.Row():
                with gr.Column(scale=2):
                    tema_input = gr.Textbox(
                        label="Tema do Poema",
                        placeholder="Ex: A beleza do pôr do sol, saudade, amor impossível...",
                        lines=3,
                        max_lines=5
                    )

                    estilo_input = gr.Dropdown(
                        label="Estilo do Poema",
                        choices=[
                            "Livre (sem estrutura específica)",
                            "Romântico",
                            "Haicai (estilo japonês)",
                            "Soneto",
                            "Moderno",
                            "Clássico",
                            "Melancólico",
                            "Alegre e descontraído"
                        ],
                        value="Livre (sem estrutura específica)"
                    )

                    tamanho_input = gr.Radio(
                        label="Tamanho do Poema",
                        choices=["Curto (1-2 estrofes)", "Médio (3-4 estrofes)", "Longo (5+ estrofes)"],
                        value="Médio (3-4 estrofes)"
                    )

                with gr.Column(scale=1):
                    gr.Markdown("### ⚙️ Configurações Avançadas")

                    temperatura_input = gr.Slider(
                        minimum=0,
                        maximum=100,
                        value=70,
                        step=1,
                        label="Criatividade",
                        info="Maior = mais criativo e variado"
                    )

                    gr.Markdown("---")
                    gr.Markdown("### 💡 Exemplos de Temas")
                    gr.Markdown("""
                    - O som da chuva à noite
                    - Um gato observando a lua
                    - A solidão na cidade grande
                    - Memórias de infância
                    """)

            with gr.Row(elem_classes=["button-row"]):
                gerar_btn = gr.Button("🚀 Gerar Poema", variant="primary", elem_classes=["generate-btn"])
                limpar_btn = gr.Button("🗑️ Limpar", variant="secondary", elem_classes=["limpar-btn", "generate-btn"])

        # ABA 2: Resultado do Poema
        with gr.Tab("📜 Seu Poema", id=1):
            gr.Markdown("### ✨ Seu poema foi gerado!")

            poema_output = gr.Textbox(
                label="",
                lines=15,
                max_lines=20,
                show_label=False,
                interactive=True,
                placeholder="Seu poema aparecerá aqui após a geração..."
            )

            gr.Markdown("---")

            # Botão para mostrar/ocultar refinamentos
            with gr.Row():
                mostrar_refinamentos_btn = gr.Button("➕ Adicionar Refinamentos", variant="secondary")

            # Container de refinamentos (inicialmente oculto)
            refinamentos_container = gr.Column(visible=False)

            with refinamentos_container:
                gr.Markdown("### ✂️ Refinamentos")

                # Container dinâmico para múltiplos blocos
                refinamentos_accordion = gr.Accordion("Refinamentos Adicionados", open=True)

                with refinamentos_accordion:
                    refinamentos_display = gr.Markdown("*Nenhum refinamento adicionado ainda*")

                gr.Markdown("#### Adicionar Novo Refinamento")

                with gr.Row(elem_classes=["refinamento-box"]):
                    with gr.Column():
                        acao_temp = gr.Radio(
                            label="Ação",
                            choices=["Adicionar", "Editar", "Remover"],
                            value="Editar"
                        )
                        alvo_temp = gr.Textbox(
                            label="Trecho/Alvo",
                            placeholder="Ex: 'A terceira estrofe', 'A palavra vento'",
                            lines=1
                        )
                        novo_conteudo_temp = gr.Textbox(
                            label="Novo Conteúdo",
                            placeholder="Digite o novo texto",
                            lines=2
                        )

                with gr.Row():
                    adicionar_refinamento_btn = gr.Button("➕ Adicionar à Lista", variant="secondary")
                    limpar_refinamentos_btn = gr.Button("🗑️ Limpar Todos", variant="secondary")

                refinar_btn = gr.Button("🔄 Aplicar Refinamentos", variant="primary", elem_classes=["generate-btn"])

            with gr.Row():
                copiar_btn = gr.Button("📋 Copiar Poema", variant="secondary")
                novo_btn = gr.Button("✏️ Criar Novo Poema", variant="primary")


    # --- Funções de Controle ---
    def toggle_refinamentos():
        return gr.Column(visible=True)


    def adicionar_refinamento(refinamentos, acao, alvo, novo_conteudo):
        if not alvo:
            return refinamentos, formatar_refinamentos(refinamentos), "", "", ""

        refinamentos.append({
            'acao': acao,
            'alvo': alvo,
            'novo_conteudo': novo_conteudo
        })

        return refinamentos, formatar_refinamentos(refinamentos), "Editar", "", ""


    def limpar_refinamentos():
        return [], "*Nenhum refinamento adicionado ainda*"


    def formatar_refinamentos(refinamentos):
        if not refinamentos:
            return "*Nenhum refinamento adicionado ainda*"

        texto = ""
        for i, ref in enumerate(refinamentos, 1):
            texto += f"**{i}. {ref['acao']}**: {ref['alvo']}"
            if ref['novo_conteudo']:
                texto += f" → *{ref['novo_conteudo']}*"
            texto += "\n\n"
        return texto


    def limpar_campos():
        return "", "Livre (sem estrutura específica)", "Médio (3-4 estrofes)", 70, "", [], None


    def voltar_criar():
        return gr.Tabs(selected=0)


    # --- Conectar Eventos ---
    # Geração inicial (cria novo chat)
    gerar_btn.click(
        fn=lambda t, e, tam, temp, p, r, c: gerar_poema(t, e, tam, temp, p, [], c),
        inputs=[tema_input, estilo_input, tamanho_input, temperatura_input, poema_state, refinamentos_state,
                chat_state],
        outputs=[poema_output, poema_state, tabs, chat_state]
    )

    # Toggle refinamentos
    mostrar_refinamentos_btn.click(
        fn=toggle_refinamentos,
        outputs=[refinamentos_container]
    )

    # Adicionar refinamento à lista
    adicionar_refinamento_btn.click(
        fn=adicionar_refinamento,
        inputs=[refinamentos_state, acao_temp, alvo_temp, novo_conteudo_temp],
        outputs=[refinamentos_state, refinamentos_display, acao_temp, alvo_temp, novo_conteudo_temp]
    )

    # Limpar refinamentos
    limpar_refinamentos_btn.click(
        fn=limpar_refinamentos,
        outputs=[refinamentos_state, refinamentos_display]
    )

    # Aplicar refinamentos (usa o chat existente)
    refinar_btn.click(
        fn=gerar_poema,
        inputs=[tema_input, estilo_input, tamanho_input, temperatura_input, poema_state, refinamentos_state,
                chat_state],
        outputs=[poema_output, poema_state, tabs, chat_state]
    ).then(
        fn=limpar_refinamentos,
        outputs=[refinamentos_state, refinamentos_display]
    )

    # Atualizar state quando editar manualmente
    poema_output.change(fn=lambda x: x, inputs=poema_output, outputs=poema_state)

    # Outros botões
    limpar_btn.click(fn=limpar_campos, outputs=[tema_input, estilo_input, tamanho_input, temperatura_input, poema_state,
                                                refinamentos_state, chat_state])
    novo_btn.click(fn=voltar_criar, outputs=tabs)

# --- Lançar Interface ---
demo.launch(inline=True, share=True)